<a href="https://colab.research.google.com/github/falcowinkler/tweet-classification/blob/master/TwitterFakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')

def tokenize(line):
    """Takes a tweet and turns it into whitespace seperated string of tokens"""
    line = re.sub("[^a-zA-Z]", ' ', line) # replace non digits
    words = line.split()
    words = [word for word in words if not word in set(stopwords.words('english'))]
    line = ' '.join(words)
    return line
    
def get_dataset(df):
    df.text=df.text.map(tokenize)
    print(df.text)
    tokenizer = Tokenizer(split=' ', num_words=1000)
    tokenizer.fit_on_texts(df.text)
    encoded_urls = tokenizer.texts_to_matrix(df.text, mode='count')
    labels = np.array(df.target)
    return encoded_urls, labels

df = pd.read_csv('train.csv', sep=',')
features, labels = get_dataset(df)

encoded_words_train , encoded_words_test , labels_train , labels_test = \
train_test_split(features,labels,test_size=0.20, random_state=55, shuffle =True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0        Our Deeds Reason earthquake May ALLAH Forgive us
1                   Forest fire near La Ronge Sask Canada
2       All residents asked shelter place notified off...
3       people receive wildfires evacuation orders Cal...
4       Just got sent photo Ruby Alaska smoke wildfire...
                              ...                        
7608    Two giant cranes holding bridge collapse nearb...
7609    aria ahrary TheTawniest The control wild fires...
7610         M UTC km S Volcano Hawaii http co zDtoyd EbJ
7611    Police investigating e bike collided car Littl...
7612    The Latest More Homes Razed Northern Californi...
Name: text, Length: 7613, dtype: object


In [27]:
lg = LogisticRegression(max_iter=200)
lg.fit(encoded_words_train,labels_train)
print(lg.score(encoded_words_test, labels_test))

0.7793827971109653


In [29]:
model = keras.Sequential([
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])
model.fit(encoded_words_train, labels_train, epochs=100, batch_size=64)

print('Done training!')
model.evaluate(encoded_words_test, labels_test)

Epoch 1/100
96/96 [==============================] - 0s 2ms/step - loss: 0.2148 - accuracy: 0.7067
Epoch 2/100
96/96 [==============================] - 0s 2ms/step - loss: 0.1521 - accuracy: 0.8080
Epoch 3/100
96/96 [==============================] - 0s 2ms/step - loss: 0.1291 - accuracy: 0.8330
Epoch 4/100
96/96 [==============================] - 0s 2ms/step - loss: 0.1159 - accuracy: 0.8504
Epoch 5/100
96/96 [==============================] - 0s 2ms/step - loss: 0.1100 - accuracy: 0.8571
Epoch 6/100
96/96 [==============================] - 0s 2ms/step - loss: 0.1028 - accuracy: 0.8681
Epoch 7/100
96/96 [==============================] - 0s 2ms/step - loss: 0.0980 - accuracy: 0.8752
Epoch 8/100
96/96 [==============================] - 0s 2ms/step - loss: 0.0939 - accuracy: 0.8800
Epoch 9/100
96/96 [==============================] - 0s 2ms/step - loss: 0.0886 - accuracy: 0.8941
Epoch 10/100
96/96 [==============================] - 0s 2ms/step - loss: 0.0851 - accuracy: 0.9002
Epoch 11/

[0.20200031995773315, 0.7524622678756714]